In [1]:
from itertools import count

In [2]:
import gym
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
env = gym.make('LunarLander-v2')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
class PolicyNetwork(tf.keras.Model):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, input_shape=(8,))
        self.dense2 = tf.keras.layers.Dense(units=4)
        
        self.saved_log_probs = []
        self.rewards = []

    def call(self, input):
        """Run the model."""
        result = self.dense1(input)
        result = self.dense2(result)
        result = tf.nn.softmax(result)
        return result

pi = PolicyNetwork()
optimizer = tf.train.AdamOptimizer(learning_rate=3e-4)

In [12]:
def select_action(state):
    with tf.GradientTape() as tape:
        state = tf.expand_dims(tf.constant(state, dtype=tf.float32), 0)
        probs = pi(state)
        m = tf.distributions.Categorical(probs)
        action = m.sample()
        pi.saved_log_probs.append(m.log_prob(action))
    return action.numpy()[0]


def finish_episode():
    with tf.GradientTape() as tape:
        R = 0
        policy_loss = []
        rewards = []
        for r in pi.rewards[::-1]:
            R = r + gamma * R
            rewards.insert(0, R)
        rewards = tf.constant(rewards)
        rew_mean, rew_var = tf.nn.moments(rewards, axes=[0])
        rewards = (rewards - rew_mean) / (rew_var + eps)
        for log_prob, reward in zip(pi.saved_log_probs, rewards):
            policy_loss.append(-log_prob * reward)
        policy_loss = tf.reduce_sum(policy_loss, 0)
    
    grads = tape.gradient(policy_loss, pi.trainable_variables)
    print(grads)
    optimizer.apply_gradients(zip(grads, pi.trainable_variables),
                            global_step=tf.train.get_or_create_global_step())

    del pi.rewards[:]
    del pi.saved_log_probs[:]


render = True
log_interval = 100
gamma = 0.99
eps = 1e-9
def main():
    running_reward = 10
    for i_episode in count(1):
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if render:
                env.render()
            pi.rewards.append(reward)
            if done:
                break

        running_reward = running_reward * 0.99 + t * 0.01
        finish_episode()
        if i_episode % log_interval == 0:
            print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
                i_episode, t, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break


if __name__ == '__main__':
    main()

[None, None, None, None]


ValueError: No gradients provided for any variable: ['<tensorflow.python.training.optimizer._DenseResourceVariableProcessor object at 0x7f41d5854f28>', '<tensorflow.python.training.optimizer._DenseResourceVariableProcessor object at 0x7f41d5854da0>', '<tensorflow.python.training.optimizer._DenseResourceVariableProcessor object at 0x7f41d5854d30>', '<tensorflow.python.training.optimizer._DenseResourceVariableProcessor object at 0x7f41d5854d68>'].

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])
a.numpy()